# Get Multiple Twitter Users Data
## This script uses Twitter API v2 to query Twitter for one or more user's data.

#### Useful for collecting data on user Followers or users Following. 

In [ ]:
# -*- coding: utf-8 -*-
"""
Created on 2022-12-14

@author: jsale
"""

In [ ]:
import requests
import os
import json
import glob
import csv
import sys
import json
import datetime as dt
from pprint import pprint

def create_url(usernames):
    # User fields are adjustable, options include:
    # created_at, description, entities, id, location, name,
    # pinned_tweet_id, profile_image_url, protected,
    # public_metrics, url, username, verified, and withheld
    user_fields = "user.fields=description,created_at,public_metrics"
    url = "https://api.twitter.com/2/users/by?{}&{}".format(usernames, user_fields)
    return url

def bearer_oauth(r):
    r.headers["Authorization"] = "Bearer <<Your Bearer Token Here>>"
    r.headers["User-Agent"] = "v2SampledStreamPython"
    return r

def connect_to_endpoint(url):
    response = requests.request("GET", url, auth=bearer_oauth,)
    print(response.status_code)
    if response.status_code != 200:
        raise Exception(
            "Request returned an error: {} {}".format(
                response.status_code, response.text
            )
        )
    return response.json()

def main():
    # Example lists containing twitter users
    test = ['CBSMornings','nytimes','TechCrunch']
    # Set the twitter_user_array equal to one of the example lists
    twitter_user_array = test
    
    # Open output file and write the header row
    outfile = "twitter_user_data.csv"
    f_out = open(outfile, 'w', newline='', encoding='utf-8')
    rowwriter = csv.writer(f_out, delimiter=',')
    outputrow = ['username','created_at','id','name','followers_count','following_count','listed_count','tweet_count','description']
    rowwriter.writerow(outputrow)
    
    # Loop through the twitter_user_array, query user data, and save to a file
    for i in range(len(twitter_user_array)):
        user = "usernames="+twitter_user_array[i]
        url = create_url(user)
        json_response = connect_to_endpoint(url)
        username = json_response['data'][0]['username']
        created_at = json_response['data'][0]['created_at']
        id = json_response['data'][0]['id']
        name = json_response['data'][0]['name']
        followers_count = json_response['data'][0]['public_metrics']['followers_count']
        following_count = json_response['data'][0]['public_metrics']['following_count']
        listed_count = json_response['data'][0]['public_metrics']['listed_count']
        tweet_count = json_response['data'][0]['public_metrics']['tweet_count']
        description = json_response['data'][0]['description']
        
        # Optional print statement to track progress
        print("#" + str(i) + " " + user + " id:" + str(id))
        data = json_response
        
        # Optional print statement for user data
        print(data)
        
        # Define output string and write to file
        outputrow = [username,created_at,id,name,followers_count,following_count,listed_count,tweet_count,description]
        rowwriter.writerow(outputrow)
    f_out.close()

if __name__ == "__main__":
    main()
